In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}



def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [3]:
import time


def specialModulus(a):
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        term=d*8^o2     
        ans=ans+term*x^a
    return D(ans)


def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        
        term=d*8^o2*x^a        
        ans=ans+term
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
start=time.time()
for a in [1..1000]: #default precision must be > max a.
    c=p2RecipPowerConstantTerm(a)
    o2=order(2,c)
    ds=digitsum(2,a)
    finish=time.time()
    test=o2-3*ds
    tests=tests+[test]
    print((a,test))
print(tests)

(1, 0)
(2, 0)
(3, 0)
(4, 0)
(5, 0)
(6, 0)
(7, 0)
(8, 0)
(9, 0)
(10, 0)
(11, 0)
(12, 0)
(13, 0)
(14, 0)
(15, 0)
(16, 0)
(17, 0)
(18, 0)
(19, 0)
(20, 0)
(21, 0)
(22, 0)
(23, 0)
(24, 0)
(25, 0)
(26, 0)
(27, 0)
(28, 0)
(29, 0)
(30, 0)
(31, 0)
(32, 0)
(33, 0)
(34, 0)
(35, 0)
(36, 0)
(37, 0)
(38, 0)
(39, 0)
(40, 0)
(41, 0)
(42, 0)
(43, 0)
(44, 0)
(45, 0)
(46, 0)
(47, 0)
(48, 0)
(49, 0)
(50, 0)
(51, 0)
(52, 0)
(53, 0)
(54, 0)
(55, 0)
(56, 0)
(57, 0)
(58, 0)
(59, 0)
(60, 0)
(61, 0)
(62, 0)
(63, 0)
(64, 0)
(65, 0)
(66, 0)
(67, 0)
(68, 0)
(69, 0)
(70, 0)
(71, 0)
(72, 0)
(73, 0)
(74, 0)
(75, 0)
(76, 0)
(77, 0)
(78, 0)
(79, 0)
(80, 0)
(81, 0)
(82, 0)
(83, 0)
(84, 0)
(85, 0)
(86, 0)
(87, 0)
(88, 0)
(89, 0)
(90, 0)
(91, 0)
(92, 0)
(93, 0)
(94, 0)
(95, 0)
(96, 0)
(97, 0)
(98, 0)
(99, 0)
(100, 0)
(101, 0)
(102, 0)
(103, 0)
(104, 0)
(105, 0)
(106, 0)
(107, 0)
(108, 0)
(109, 0)
(110, 0)
(111, 0)
(112, 0)
(113, 0)
(114, 0)
(115, 0)
(116, 0)
(117, 0)
(118, 0)
(119, 0)
(120, 0)
(121, 0)
(122, 0)
(123, 0)
(

(924, 0)
(925, 0)
(926, 0)
(927, 0)
(928, 0)
(929, 0)
(930, 0)
(931, 0)
(932, 0)
(933, 0)
(934, 0)
(935, 0)
(936, 0)
(937, 0)
(938, 0)
(939, 0)
(940, 0)
(941, 0)
(942, 0)
(943, 0)
(944, 0)
(945, 0)
(946, 0)
(947, 0)
(948, 0)
(949, 0)
(950, 0)
(951, 0)
(952, 0)
(953, 0)
(954, 0)
(955, 0)
(956, 0)
(957, 0)
(958, 0)
(959, 0)
(960, 0)
(961, 0)
(962, 0)
(963, 0)
(964, 0)
(965, 0)
(966, 0)
(967, 0)
(968, 0)
(969, 0)
(970, 0)
(971, 0)
(972, 0)
(973, 0)
(974, 0)
(975, 0)
(976, 0)
(977, 0)
(978, 0)
(979, 0)
(980, 0)
(981, 0)
(982, 0)
(983, 0)
(984, 0)
(985, 0)
(986, 0)
(987, 0)
(988, 0)
(989, 0)
(990, 0)
(991, 0)
(992, 0)
(993, 0)
(994, 0)
(995, 0)
(996, 0)
(997, 0)
(998, 0)
(999, 0)
(1000, 0)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [4]:
import time
import random

def specialModulus(a):
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        
        term=d*8^o2+sm*r      
        ans=ans+term*x^a
    return D(ans)


def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        
        term=d*8^o2*x^a        
        ans=ans+term
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

for b in [1..5]:
    tests=[]
    for a in [1..1000]:
        c=p2RecipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
    
        test=o2-3*ds
        tests=tests+[test]
        print((b,a,test))
print(tests)

(1, 1, 0)
(1, 2, 0)
(1, 3, 0)
(1, 4, 0)
(1, 5, 0)
(1, 6, 0)
(1, 7, 0)
(1, 8, 0)
(1, 9, 0)
(1, 10, 0)
(1, 11, 0)
(1, 12, 0)
(1, 13, 0)
(1, 14, 0)
(1, 15, 0)
(1, 16, 0)
(1, 17, 0)
(1, 18, 0)
(1, 19, 0)
(1, 20, 0)
(1, 21, 0)
(1, 22, 0)
(1, 23, 0)
(1, 24, 0)
(1, 25, 0)
(1, 26, 0)
(1, 27, 0)
(1, 28, 0)
(1, 29, 0)
(1, 30, 0)
(1, 31, 0)
(1, 32, 0)
(1, 33, 0)
(1, 34, 0)
(1, 35, 0)
(1, 36, 0)
(1, 37, 0)
(1, 38, 0)
(1, 39, 0)
(1, 40, 0)
(1, 41, 0)
(1, 42, 0)
(1, 43, 0)
(1, 44, 0)
(1, 45, 0)
(1, 46, 0)
(1, 47, 0)
(1, 48, 0)
(1, 49, 0)
(1, 50, 0)
(1, 51, 0)
(1, 52, 0)
(1, 53, 0)
(1, 54, 0)
(1, 55, 0)
(1, 56, 0)
(1, 57, 0)
(1, 58, 0)
(1, 59, 0)
(1, 60, 0)
(1, 61, 0)
(1, 62, 0)
(1, 63, 0)
(1, 64, 0)
(1, 65, 0)
(1, 66, 0)
(1, 67, 0)
(1, 68, 0)
(1, 69, 0)
(1, 70, 0)
(1, 71, 0)
(1, 72, 0)
(1, 73, 0)
(1, 74, 0)
(1, 75, 0)
(1, 76, 0)
(1, 77, 0)
(1, 78, 0)
(1, 79, 0)
(1, 80, 0)
(1, 81, 0)
(1, 82, 0)
(1, 83, 0)
(1, 84, 0)
(1, 85, 0)
(1, 86, 0)
(1, 87, 0)
(1, 88, 0)
(1, 89, 0)
(1, 90, 0)
(1, 91, 0)
(1, 92, 

(1, 693, 0)
(1, 694, 0)
(1, 695, 0)
(1, 696, 0)
(1, 697, 0)
(1, 698, 0)
(1, 699, 0)
(1, 700, 0)
(1, 701, 0)
(1, 702, 0)
(1, 703, 0)
(1, 704, 0)
(1, 705, 0)
(1, 706, 0)
(1, 707, 0)
(1, 708, 0)
(1, 709, 0)
(1, 710, 0)
(1, 711, 0)
(1, 712, 0)
(1, 713, 0)
(1, 714, 0)
(1, 715, 0)
(1, 716, 0)
(1, 717, 0)
(1, 718, 0)
(1, 719, 0)
(1, 720, 0)
(1, 721, 0)
(1, 722, 0)
(1, 723, 0)
(1, 724, 0)
(1, 725, 0)
(1, 726, 0)
(1, 727, 0)
(1, 728, 0)
(1, 729, 0)
(1, 730, 0)
(1, 731, 0)
(1, 732, 0)
(1, 733, 0)
(1, 734, 0)
(1, 735, 0)
(1, 736, 0)
(1, 737, 0)
(1, 738, 0)
(1, 739, 0)
(1, 740, 0)
(1, 741, 0)
(1, 742, 0)
(1, 743, 0)
(1, 744, 0)
(1, 745, 0)
(1, 746, 0)
(1, 747, 0)
(1, 748, 0)
(1, 749, 0)
(1, 750, 0)
(1, 751, 0)
(1, 752, 0)
(1, 753, 0)
(1, 754, 0)
(1, 755, 0)
(1, 756, 0)
(1, 757, 0)
(1, 758, 0)
(1, 759, 0)
(1, 760, 0)
(1, 761, 0)
(1, 762, 0)
(1, 763, 0)
(1, 764, 0)
(1, 765, 0)
(1, 766, 0)
(1, 767, 0)
(1, 768, 0)
(1, 769, 0)
(1, 770, 0)
(1, 771, 0)
(1, 772, 0)
(1, 773, 0)
(1, 774, 0)
(1, 775, 0)
(1, 

(2, 385, 0)
(2, 386, 0)
(2, 387, 0)
(2, 388, 0)
(2, 389, 0)
(2, 390, 0)
(2, 391, 0)
(2, 392, 0)
(2, 393, 0)
(2, 394, 0)
(2, 395, 0)
(2, 396, 0)
(2, 397, 0)
(2, 398, 0)
(2, 399, 0)
(2, 400, 0)
(2, 401, 0)
(2, 402, 0)
(2, 403, 0)
(2, 404, 0)
(2, 405, 0)
(2, 406, 0)
(2, 407, 0)
(2, 408, 0)
(2, 409, 0)
(2, 410, 0)
(2, 411, 0)
(2, 412, 0)
(2, 413, 0)
(2, 414, 0)
(2, 415, 0)
(2, 416, 0)
(2, 417, 0)
(2, 418, 0)
(2, 419, 0)
(2, 420, 0)
(2, 421, 0)
(2, 422, 0)
(2, 423, 0)
(2, 424, 0)
(2, 425, 0)
(2, 426, 0)
(2, 427, 0)
(2, 428, 0)
(2, 429, 0)
(2, 430, 0)
(2, 431, 0)
(2, 432, 0)
(2, 433, 0)
(2, 434, 0)
(2, 435, 0)
(2, 436, 0)
(2, 437, 0)
(2, 438, 0)
(2, 439, 0)
(2, 440, 0)
(2, 441, 0)
(2, 442, 0)
(2, 443, 0)
(2, 444, 0)
(2, 445, 0)
(2, 446, 0)
(2, 447, 0)
(2, 448, 0)
(2, 449, 0)
(2, 450, 0)
(2, 451, 0)
(2, 452, 0)
(2, 453, 0)
(2, 454, 0)
(2, 455, 0)
(2, 456, 0)
(2, 457, 0)
(2, 458, 0)
(2, 459, 0)
(2, 460, 0)
(2, 461, 0)
(2, 462, 0)
(2, 463, 0)
(2, 464, 0)
(2, 465, 0)
(2, 466, 0)
(2, 467, 0)
(2, 

(3, 82, 0)
(3, 83, 0)
(3, 84, 0)
(3, 85, 0)
(3, 86, 0)
(3, 87, 0)
(3, 88, 0)
(3, 89, 0)
(3, 90, 0)
(3, 91, 0)
(3, 92, 0)
(3, 93, 0)
(3, 94, 0)
(3, 95, 0)
(3, 96, 0)
(3, 97, 0)
(3, 98, 0)
(3, 99, 0)
(3, 100, 0)
(3, 101, 0)
(3, 102, 0)
(3, 103, 0)
(3, 104, 0)
(3, 105, 0)
(3, 106, 0)
(3, 107, 0)
(3, 108, 0)
(3, 109, 0)
(3, 110, 0)
(3, 111, 0)
(3, 112, 0)
(3, 113, 0)
(3, 114, 0)
(3, 115, 0)
(3, 116, 0)
(3, 117, 0)
(3, 118, 0)
(3, 119, 0)
(3, 120, 0)
(3, 121, 0)
(3, 122, 0)
(3, 123, 0)
(3, 124, 0)
(3, 125, 0)
(3, 126, 0)
(3, 127, 0)
(3, 128, 0)
(3, 129, 0)
(3, 130, 0)
(3, 131, 0)
(3, 132, 0)
(3, 133, 0)
(3, 134, 0)
(3, 135, 0)
(3, 136, 0)
(3, 137, 0)
(3, 138, 0)
(3, 139, 0)
(3, 140, 0)
(3, 141, 0)
(3, 142, 0)
(3, 143, 0)
(3, 144, 0)
(3, 145, 0)
(3, 146, 0)
(3, 147, 0)
(3, 148, 0)
(3, 149, 0)
(3, 150, 0)
(3, 151, 0)
(3, 152, 0)
(3, 153, 0)
(3, 154, 0)
(3, 155, 0)
(3, 156, 0)
(3, 157, 0)
(3, 158, 0)
(3, 159, 0)
(3, 160, 0)
(3, 161, 0)
(3, 162, 0)
(3, 163, 0)
(3, 164, 0)
(3, 165, 0)
(3, 166, 0

(3, 767, 0)
(3, 768, 0)
(3, 769, 0)
(3, 770, 0)
(3, 771, 0)
(3, 772, 0)
(3, 773, 0)
(3, 774, 0)
(3, 775, 0)
(3, 776, 0)
(3, 777, 0)
(3, 778, 0)
(3, 779, 0)
(3, 780, 0)
(3, 781, 0)
(3, 782, 0)
(3, 783, 0)
(3, 784, 0)
(3, 785, 0)
(3, 786, 0)
(3, 787, 0)
(3, 788, 0)
(3, 789, 0)
(3, 790, 0)
(3, 791, 0)
(3, 792, 0)
(3, 793, 0)
(3, 794, 0)
(3, 795, 0)
(3, 796, 0)
(3, 797, 0)
(3, 798, 0)
(3, 799, 0)
(3, 800, 0)
(3, 801, 0)
(3, 802, 0)
(3, 803, 0)
(3, 804, 0)
(3, 805, 0)
(3, 806, 0)
(3, 807, 0)
(3, 808, 0)
(3, 809, 0)
(3, 810, 0)
(3, 811, 0)
(3, 812, 0)
(3, 813, 0)
(3, 814, 0)
(3, 815, 0)
(3, 816, 0)
(3, 817, 0)
(3, 818, 0)
(3, 819, 0)
(3, 820, 0)
(3, 821, 0)
(3, 822, 0)
(3, 823, 0)
(3, 824, 0)
(3, 825, 0)
(3, 826, 0)
(3, 827, 0)
(3, 828, 0)
(3, 829, 0)
(3, 830, 0)
(3, 831, 0)
(3, 832, 0)
(3, 833, 0)
(3, 834, 0)
(3, 835, 0)
(3, 836, 0)
(3, 837, 0)
(3, 838, 0)
(3, 839, 0)
(3, 840, 0)
(3, 841, 0)
(3, 842, 0)
(3, 843, 0)
(3, 844, 0)
(3, 845, 0)
(3, 846, 0)
(3, 847, 0)
(3, 848, 0)
(3, 849, 0)
(3, 

(4, 460, 0)
(4, 461, 0)
(4, 462, 0)
(4, 463, 0)
(4, 464, 0)
(4, 465, 0)
(4, 466, 0)
(4, 467, 0)
(4, 468, 0)
(4, 469, 0)
(4, 470, 0)
(4, 471, 0)
(4, 472, 0)
(4, 473, 0)
(4, 474, 0)
(4, 475, 0)
(4, 476, 0)
(4, 477, 0)
(4, 478, 0)
(4, 479, 0)
(4, 480, 0)
(4, 481, 0)
(4, 482, 0)
(4, 483, 0)
(4, 484, 0)
(4, 485, 0)
(4, 486, 0)
(4, 487, 0)
(4, 488, 0)
(4, 489, 0)
(4, 490, 0)
(4, 491, 0)
(4, 492, 0)
(4, 493, 0)
(4, 494, 0)
(4, 495, 0)
(4, 496, 0)
(4, 497, 0)
(4, 498, 0)
(4, 499, 0)
(4, 500, 0)
(4, 501, 0)
(4, 502, 0)
(4, 503, 0)
(4, 504, 0)
(4, 505, 0)
(4, 506, 0)
(4, 507, 0)
(4, 508, 0)
(4, 509, 0)
(4, 510, 0)
(4, 511, 0)
(4, 512, 0)
(4, 513, 0)
(4, 514, 0)
(4, 515, 0)
(4, 516, 0)
(4, 517, 0)
(4, 518, 0)
(4, 519, 0)
(4, 520, 0)
(4, 521, 0)
(4, 522, 0)
(4, 523, 0)
(4, 524, 0)
(4, 525, 0)
(4, 526, 0)
(4, 527, 0)
(4, 528, 0)
(4, 529, 0)
(4, 530, 0)
(4, 531, 0)
(4, 532, 0)
(4, 533, 0)
(4, 534, 0)
(4, 535, 0)
(4, 536, 0)
(4, 537, 0)
(4, 538, 0)
(4, 539, 0)
(4, 540, 0)
(4, 541, 0)
(4, 542, 0)
(4, 

(5, 159, 0)
(5, 160, 0)
(5, 161, 0)
(5, 162, 0)
(5, 163, 0)
(5, 164, 0)
(5, 165, 0)
(5, 166, 0)
(5, 167, 0)
(5, 168, 0)
(5, 169, 0)
(5, 170, 0)
(5, 171, 0)
(5, 172, 0)
(5, 173, 0)
(5, 174, 0)
(5, 175, 0)
(5, 176, 0)
(5, 177, 0)
(5, 178, 0)
(5, 179, 0)
(5, 180, 0)
(5, 181, 0)
(5, 182, 0)
(5, 183, 0)
(5, 184, 0)
(5, 185, 0)
(5, 186, 0)
(5, 187, 0)
(5, 188, 0)
(5, 189, 0)
(5, 190, 0)
(5, 191, 0)
(5, 192, 0)
(5, 193, 0)
(5, 194, 0)
(5, 195, 0)
(5, 196, 0)
(5, 197, 0)
(5, 198, 0)
(5, 199, 0)
(5, 200, 0)
(5, 201, 0)
(5, 202, 0)
(5, 203, 0)
(5, 204, 0)
(5, 205, 0)
(5, 206, 0)
(5, 207, 0)
(5, 208, 0)
(5, 209, 0)
(5, 210, 0)
(5, 211, 0)
(5, 212, 0)
(5, 213, 0)
(5, 214, 0)
(5, 215, 0)
(5, 216, 0)
(5, 217, 0)
(5, 218, 0)
(5, 219, 0)
(5, 220, 0)
(5, 221, 0)
(5, 222, 0)
(5, 223, 0)
(5, 224, 0)
(5, 225, 0)
(5, 226, 0)
(5, 227, 0)
(5, 228, 0)
(5, 229, 0)
(5, 230, 0)
(5, 231, 0)
(5, 232, 0)
(5, 233, 0)
(5, 234, 0)
(5, 235, 0)
(5, 236, 0)
(5, 237, 0)
(5, 238, 0)
(5, 239, 0)
(5, 240, 0)
(5, 241, 0)
(5, 

(5, 842, 0)
(5, 843, 0)
(5, 844, 0)
(5, 845, 0)
(5, 846, 0)
(5, 847, 0)
(5, 848, 0)
(5, 849, 0)
(5, 850, 0)
(5, 851, 0)
(5, 852, 0)
(5, 853, 0)
(5, 854, 0)
(5, 855, 0)
(5, 856, 0)
(5, 857, 0)
(5, 858, 0)
(5, 859, 0)
(5, 860, 0)
(5, 861, 0)
(5, 862, 0)
(5, 863, 0)
(5, 864, 0)
(5, 865, 0)
(5, 866, 0)
(5, 867, 0)
(5, 868, 0)
(5, 869, 0)
(5, 870, 0)
(5, 871, 0)
(5, 872, 0)
(5, 873, 0)
(5, 874, 0)
(5, 875, 0)
(5, 876, 0)
(5, 877, 0)
(5, 878, 0)
(5, 879, 0)
(5, 880, 0)
(5, 881, 0)
(5, 882, 0)
(5, 883, 0)
(5, 884, 0)
(5, 885, 0)
(5, 886, 0)
(5, 887, 0)
(5, 888, 0)
(5, 889, 0)
(5, 890, 0)
(5, 891, 0)
(5, 892, 0)
(5, 893, 0)
(5, 894, 0)
(5, 895, 0)
(5, 896, 0)
(5, 897, 0)
(5, 898, 0)
(5, 899, 0)
(5, 900, 0)
(5, 901, 0)
(5, 902, 0)
(5, 903, 0)
(5, 904, 0)
(5, 905, 0)
(5, 906, 0)
(5, 907, 0)
(5, 908, 0)
(5, 909, 0)
(5, 910, 0)
(5, 911, 0)
(5, 912, 0)
(5, 913, 0)
(5, 914, 0)
(5, 915, 0)
(5, 916, 0)
(5, 917, 0)
(5, 918, 0)
(5, 919, 0)
(5, 920, 0)
(5, 921, 0)
(5, 922, 0)
(5, 923, 0)
(5, 924, 0)
(5, 